In [1]:
#Importando os e glob para juntar arquvos csv
import os
import glob
#Importando Pandas e Numpy
import pandas as pd
import numpy as np

In [2]:
#juntar arquivos csv deu muito erro de formacao de dados. 
#O padrao é ler em df já como os tipos de dados corretos e depois contatenar.
#Prestacao_Conta_SIGPC_2016
#Prestacao_Conta_SIGPC_2017
#Prestacao_Conta_SIGPC_2018
#Prestacao_Conta_SIGPC_2019
#Prestacao_Conta_SIGPC_2020

#set working directory
os.chdir("D:\etl_cecampe_ne_arquivos\Olinda_PDDE\Prestacao_Conta_SIGPC")

#find all csv files in the folder
#use glob pattern matching -> extension = 'csv'
#save result in list -> all_filenames
file_extension = '.csv'
all_filenames = [i for i in glob.glob(f"*{file_extension}")]

df_concat = pd.concat([pd.read_csv(file,
                             converters={'nu_cnpj_uex': lambda x: str(x),
                             'no_razao_social_uex': lambda x: str(x), 
                             'nu_cnpj_eex': lambda x: str(x),
                             'no_razao_social_eex': lambda x: str(x),
                             'an_exercicio': lambda x: str(x),
                             'co_esfera_adm': lambda x: str(x),
                             'co_programa_fnde': lambda x: str(x),
                             'ds_situacao_uex': lambda x: str(x),
                             'st_ef_susp_vigente_uex': lambda x: str(x),
                             'st_uex_apta_pagamento': lambda x: str(x),
                             'st_ef_susp_vigente_eex': lambda x: str(x),
                             'st_eex_apta_pagamento': lambda x: str(x)}) for file in all_filenames]) 

#print(df_concat)

In [3]:
#Substituicoes

#Replace "null" textuais por vazios
for column in df_concat.columns:
    df_concat[column] = df_concat[column].replace("null","")

#Replace vírgulas da coluna no_razao_social pois isso cria aspas no string 
#e na importação dos dados para o postgres gera um erro
# ERROR: unterminated CSV quoted field CONTEXT:
df_concat["no_razao_social_uex"] = df_concat["no_razao_social_uex"].str.replace(',','', regex=False)    
df_concat["no_razao_social_eex"] = df_concat["no_razao_social_eex"].str.replace(',','', regex=False) 

#Replace aspas simples por vazios
df_concat["no_razao_social_uex"] = df_concat["no_razao_social_uex"].str.replace("'","", regex=False)    
df_concat["no_razao_social_eex"] = df_concat["no_razao_social_eex"].str.replace("'","", regex=False)

#Replace aspas duplas por vazios
df_concat["no_razao_social_uex"] = df_concat["no_razao_social_uex"].str.replace('"','', regex=False)    
df_concat["no_razao_social_eex"] = df_concat["no_razao_social_eex"].str.replace('"','', regex=False)

#Mais subistituicoes
df_concat["no_razao_social_uex"] = df_concat["no_razao_social_uex"].str.replace('ª','', regex=False)    
df_concat["no_razao_social_eex"] = df_concat["no_razao_social_eex"].str.replace('ª','', regex=False)

In [4]:
df_concat.to_csv("Prestacao_Conta_SIGPC_Concatenada_Formatada.csv", encoding='utf-8-sig', index=False)